# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
# IMAGE_PATH = WORKSPACE_PATH+'/images'
IMAGE_PATH = WORKSPACE_PATH+'/images/simulator_dataset_rgb'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
IMAGE_YAML_PATH = WORKSPACE_PATH+'/images/sim_training_data'


# 1. Create Label Map

In [ ]:
labels = [{'name':'Red', 'id':1}, {'name':'Green', 'id':2}, {'name':'Yellow', 'id':3}, {'name':'Unknown', 'id':4}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'} -c {ANNOTATION_PATH + '/train.csv'}

# 3. Choose Pretrained Models

In [ ]:
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_inception_v2_coco_2017_11_17.tar.gz

In [ ]:
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

# 4. Copy Downloaed Model Config to Training Folder

In [ ]:
# CUSTOM_MODEL_NAME = 'ssd_inception_carnd_tl'
CUSTOM_MODEL_NAME = 'ssd_mobilenet_carnd_tl'

In [ ]:
# !mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
# !cp {PRETRAINED_MODEL_PATH+'/ssd_inception_v2_coco_2017_11_17/ssd_inception_v2_coco.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

In [ ]:
# !mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
# !cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v1_coco_11_06_2017/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

# 5. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
CONFIG_PATH

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

# 6. Train the model

In [ ]:
!python train.py --logtostderr --train_dir={MODEL_PATH+'/'+CUSTOM_MODEL_NAME} --pipeline_config_path={CONFIG_PATH}

# 7. Export trained model to an frozen graph

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path={CONFIG_PATH} --trained_checkpoint_prefix {MODEL_PATH+'/'+CUSTOM_MODEL_NAME + '/model.ckpt-15000'} --output_directory={MODEL_PATH+'/'+CUSTOM_MODEL_NAME + '/frozen_graph_v6'}

# 8. Test frozen graph

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import sys
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
## Test model
PATH_TO_GRAPH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME + '/frozen_graph_v5/frozen_inference_graph.pb' ## load SSD trained on carla simulator images
PATH_TO_LABELS = ANNOTATION_PATH + '/label_map.pbtxt'
NUM_CLASSES = 4

In [ ]:
detection_graph = load_graph(PATH_TO_GRAPH)
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

In [ ]:
FIG_SIZE = (12, 8)
# PATH_TO_IMGS = IMAGE_PATH+'/train'
# PATH_TO_IMGS = IMAGE_PATH+'/Archive/imgs'
PATH_TO_IMGS = WORKSPACE_PATH+'/images/test'
# PATH_TO_IMGS = WORKSPACE_PATH+'/images/Archive/train'
TEST_IMGS = glob(os.path.join(PATH_TO_IMGS, r'*.png'))

# random.shuffle(TEST_IMGS)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detect_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detect_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detect_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for idx, img_path in enumerate(TEST_IMGS):
#             random.shuffle(TEST_IMGS)
            image = Image.open(img_path)
            image_np = load_image_into_numpy_array(image)
            image_expanded = np.expand_dims(image_np, axis=0)
            
            (boxes, scores, classes, num) = sess.run(
                [detect_boxes, detect_scores, detect_classes, num_detections],
                feed_dict={image_tensor: image_expanded})
            
            print('SCORES')
            print(scores[0])
            print('CLASSES')
            print(classes[0])
            print(img_path)
            
            viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np, 
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                line_thickness=8)
            plt.figure(figsize=FIG_SIZE)
            plt.imshow(image_np)
            plt.show()
            
#             if idx == 10:
#                 break